In [ ]:
#| default_exp datasources.icao
%load_ext autoreload
%autoreload 2

import sys,os
from pathlib import Path


In [ ]:
#| hide
# Insert in Path Project Directory
sys.path.insert(0, str(Path().cwd().parent))

# ICAO

> Este módulo concentra as constantes, funções de carga, processamento, mesclagem e salvamento de dados aeronáuticos provenientes do Software Frequency Finder da ICAO

In [ ]:
#| export
import os
from pathlib import Path
from typing import Iterable

import pandas as pd
from dotenv import find_dotenv, load_dotenv
from extracao.constants import PATH_NAV, PATH_COM, VOR_ILS_DME

load_dotenv(find_dotenv(), override=True)

True

## CONSTANTES


Dados para acesso aos dados do ICAO

In [ ]:
#| export
COLS_NAV = ['Frequency', 'Latitude', 'Longitude', 'Facility', 'Location', 'NS', 'WE']
COLS_COM = ['Frequency', 'CoordLat', 'CoordLong', 'DOC', 'Location', 'NS', 'WE']
UNIQUE_COLS = ['Frequency', 'Latitude', 'Longitude']


In [ ]:
#| export
def convert_latitude(
	lat: str,  # Latitude
	hemisphere: str,  # Hemisfério: N | S
) -> float:
	"""Converte a Latitude para formato decimal"""
	multiplier = 1 if hemisphere == 'N' else -1
	return multiplier * (float(lat[:2]) + float(lat[3:5]) / 60 + float(lat[6:8]) / 3600.0)


def convert_longitude(
	lon: str,  # Longitude
	hemisphere: str,  # Hemisfério: W | E
) -> float:
	"""Converte a longitude para formato decimal"""

	multiplier = 1 if hemisphere == 'E' else -1
	return multiplier * (float(lon[1:3]) + float(lon[4:6]) / 60 + float(lon[7:9]) / 3600.0)


In [ ]:
from fastcore.test import test_close

In [ ]:
lat = "25D36'01"
long = "056D06'00"
test_close(convert_latitude(lat, 'S'), -25.60027)
test_close(convert_longitude(long, 'W'), -56.10000)

In [ ]:
#|export
def _read_df(
	path: str,  # Caminho do arquivo
	usecols: Iterable[str],  # Subconjunto de colunas do arquivo
) -> pd.DataFrame:  # Dataframe formatado
	# sourcery skip: use-fstring-for-concatenation
	"""Lê o DataFrame no caminho `path`, filtra as colunas `usecols` e o retorna formatado"""
	df = pd.read_csv(path, dtype='string')[usecols]
	df.columns = COLS_NAV
	df['Latitude'] = df.apply(lambda x: convert_latitude(x['Latitude'], x['NS']), axis=1)
	df['Longitude'] = df.apply(lambda x: convert_longitude(x['Longitude'], x['WE']), axis=1)
	df['Description'] = '[ICAO] ' + df.Facility + ', ' + df.Location
	return df[['Frequency', 'Latitude', 'Longitude', 'Description']]


In [ ]:
#| export
def map_channels(
	df: pd.DataFrame,  # DataFrame dos dados de origem
	origem: str,  # Descrição da emissão a ser substituída
) -> pd.DataFrame:
	"""Mapeia os canais contidos em `df` e adiciona os registros ILS/DME caso houver"""
	chs = pd.read_csv(VOR_ILS_DME, dtype='string[pyarrow]', dtype_backend='pyarrow')
	for row in df[df.Description.str.contains('ILS|DME')].itertuples():
		if not (ch := chs[(chs.VOR_ILSloc == row.Frequency)]).empty:
			for i, c in enumerate(ch.values[0][2:]):
				if pd.notna(c):
					if i == 0:
						freq_type = 'ILS glide path'
					elif i == 1:
						freq_type = 'Airbone DME'
					elif i == 2:
						freq_type = 'Ground-based DME'
					else:
						raise ValueError('No additional frequency to map on channel')
					description = f"{row.Description.replace(origem , 'DOC')} ({freq_type})"
					df.loc[len(df)] = [c, row.Latitude, row.Longitude, description]
	return df


In [ ]:
#| export    
def get_icao() -> pd.DataFrame:  # DataFrame com frequências, coordenadas e descrição das estações
	"""Lê, concatena e pós-processa os arquivos do ICAO"""
	df = pd.concat(_read_df(p, c) for p, c in zip([PATH_NAV, PATH_COM], [COLS_NAV, COLS_COM]))
	df = df.astype('string')
	return map_channels(df, 'ICAO').drop_duplicates(UNIQUE_COLS, ignore_index=True)

In [ ]:
#| eval: false
get_icao()

,Frequency,Latitude,Longitude,Description
0,113.4,-9.866666666666667,-56.1,"[ICAO] VOR/DME, ALTA FLORESTA"
1,113.2,-3.25,-52.25,"[ICAO] VOR/DME, ALTAMIRA"
2,117.5,-4.183333333333334,-69.93333333333334,"[ICAO] VOR/DME, AMAZONICA"
3,115.4,-16.25,-49.0,"[ICAO] VOR/DME, ANAPOLIS"
4,112.0,-10.983333333333333,-37.06666666666667,"[ICAO] VOR/DME, ARACAJU STA. MARIA"
...,...,...,...,...
2626,1176,-20.766666666666666,-51.55,"[DOC] VOR/DME, URUBUPUNGA CASTILHO (Ground-bas..."
2627,1082.0,-12.7,-60.083333333333336,"[DOC] VOR/DME, VILHENA (Airbone DME)"
2628,1019,-12.7,-60.083333333333336,"[DOC] VOR/DME, VILHENA (Ground-based DME)"
2629,1126.0,-20.25,-40.28333333333333,"[DOC] VOR/DME, VITORIA GOIABEIRAS (Airbone DME)"
